# Google Inception V3 Architecture

## Load Packages

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from keras.applications import inception_v3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.callbacks import ModelCheckpoint

from keras.models import Sequential,Model
from keras.layers import Activation
from keras.layers.core import Dense,Flatten,Dropout
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam, RMSprop , SGD

Using TensorFlow backend.


## Directory

In [2]:
base_dir =r'D:\datasets'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir,'validation')
test_dir = os.path.join(base_dir, 'test')

## Generate Model

In [3]:
from keras import models
from keras import layers

conv_base = inception_v3.InceptionV3(weights = 'imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential()
for layer in conv_base.layers[:-12]:
  layer.trainable=False

x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(5, activation='softmax')(x)
model = models.Model(inputs=conv_base.input, outputs=predictions)


## Image Preprocessing 

In [4]:
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                target_size=(224, 224),
                                batch_size=10,
                                class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                target_size=(224, 224),
                                batch_size=10,
                                class_mode='categorical')

Found 29978 images belonging to 5 classes.
Found 9788 images belonging to 5 classes.


In [5]:
labels=(train_generator.class_indices)
print(labels)

{'abir': 0, 'bobi': 1, 'empty': 2, 'rafi': 3, 'unknown': 4}


## Compile and Fit

In [6]:
from keras.metrics import AUC, SensitivityAtSpecificity


model.compile(loss='categorical_crossentropy',
                                optimizer=optimizers.RMSprop(lr=2e-5),
                                metrics=['acc', AUC(), SensitivityAtSpecificity(.5)])


filepath = "inception_custom_v3_weights_all.best.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose =1, save_best_only=True, mode = 'max')
callbacks_list = [checkpoint]

history = model.fit_generator(train_generator,
                                steps_per_epoch=64,
                                epochs=10, 
                                callbacks=callbacks_list, 
                                verbose =1, 
                                validation_data=validation_generator,
                                validation_steps=64)

Epoch 1/10
64/64 [==============================] - 17s 270ms/step - loss: 1.5511 - acc: 0.2891 - auc_1: 0.6256 - sensitivity_at_specificity_1: 0.7063 - val_loss: 1.2341 - val_acc: 0.5094 - val_auc_1: 0.8106 - val_sensitivity_at_specificity_1: 0.8813

Epoch 00001: val_acc improved from -inf to 0.50937, saving model to inception_custom_v3_weights_all.best.hdf5
Epoch 2/10
64/64 [==============================] - 10s 160ms/step - loss: 1.4510 - acc: 0.3875 - auc_1: 0.7057 - sensitivity_at_specificity_1: 0.7891 - val_loss: 1.6643 - val_acc: 0.6000 - val_auc_1: 0.8447 - val_sensitivity_at_specificity_1: 0.8687

Epoch 00002: val_acc improved from 0.50937 to 0.60000, saving model to inception_custom_v3_weights_all.best.hdf5
Epoch 3/10
64/64 [==============================] - 10s 163ms/step - loss: 1.3482 - acc: 0.4703 - auc_1: 0.7746 - sensitivity_at_specificity_1: 0.8453 - val_loss: 1.1982 - val_acc: 0.7297 - val_auc_1: 0.9081 - val_sensitivity_at_specificity_1: 0.9094

Epoch 00003: val_acc 

In [7]:
from keras import optimizers

test_datagen = ImageDataGenerator(rescale=1./255)



model.load_weights("inception_custom_v3_weights_all.best.hdf5")

from keras import optimizers
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=2e-5),metrics=['acc'])




test_generator = test_datagen.flow_from_directory(test_dir,target_size=(224,224),batch_size=30,class_mode=None, shuffle = False)
probabilities = model.predict_generator(test_generator, 968)
prob=probabilities[0:968,:]


Found 133 images belonging to 5 classes.


In [8]:
import pandas as pd
from sklearn.preprocessing import label_binarize
labels = np.array([0] * 242 + [1] * 242 + [2] * 242 + [3] * 242 + [4] * 242)
y_test = label_binarize(labels, classes=[0, 1, 2, 3, 4])

In [9]:
y_cal = np.argmax(prob, axis = 1)
y_score = label_binarize(y_cal, classes=[0, 1, 2, 3, 4])

In [10]:
#Confusion Matrix and test accuracy
from sklearn import metrics
from sklearn.metrics import accuracy_score

print(metrics.accuracy_score(labels,y_cal))
confusion = metrics.confusion_matrix(labels,y_cal)
print(confusion)

from pycm import ConfusionMatrix
cm = ConfusionMatrix(actual_vector=labels, predict_vector=y_cal)
print(cm)

ValueError: Found input variables with inconsistent numbers of samples: [1210, 968]

In [ ]:
#Precision recall curve. The code is mainly adapted from the following link
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score


n_classes=5
# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_score[:, i])
    average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(),
    y_score.ravel())
average_precision["micro"] = average_precision_score(y_test, y_score,
                                                     average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'
      .format(average_precision["micro"]))


### Evaluate Model

In [ ]:
# model.evaluate(validation_generator)

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                target_size=(224, 224),
                                batch_size=10,
                                class_mode='categorical')
model.evaluate(test_generator)

In [ ]:
y_true = validation_generator.classes
predictions = model.predict_generator(validation_generator)
y_pred = np.array([np.argmax(x) for x in predictions])
y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
import seaborn as sns

ax = sns.heatmap(cm, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
print(classification_report(y_true, y_pred))

### Plot Accuracy Graph and Metrices

In [ ]:
# import matplotlib.pyplot as plt

# acc = history.history['acc']
# val_acc =history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# auc = history.history['auc_2']
# val_auc = history.history['val_auc_2']



# epochs = range(1, len(acc)+1)

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.figure()
# plt.plot(epochs, loss, 'bo', label= 'Training loss')
# plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.figure()
# plt.plot(epochs, auc, 'bo', label= 'AUC')
# plt.plot(epochs, val_auc, 'b', label = 'AUC loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()

### ROC Curve

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

validation_dir=r"D:\datasets\test"
generator = ImageDataGenerator()
validation_ds = generator.flow_from_directory(validation_dir,target_size=(224, 224),batch_size=1)

# model.evaluate_generator(validation_ds)

# Get the names of the 5 classes
class_names = validation_ds.class_indices.keys()

preds = model.predict(validation_ds)
pred_classes = np.argmax(preds, axis=1)

true_classes = validation_ds.classes


def plot_heatmap(y_true, y_pred, class_names, ax, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(
        cm, 
        annot=True, 
        square=True, 
        xticklabels=class_names, 
        yticklabels=class_names,
        fmt='d', 
        cmap=plt.cm.Blues,
        cbar=False,
        ax=ax
    )
    ax.set_title(title, fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    ax.set_ylabel('True Label', fontsize=12)
    ax.set_xlabel('Predicted Label', fontsize=12)

fig, (ax1) = plt.subplots(1, 1, figsize=(10, 10))

# plot_heatmap(true_classes, scratch_pred_classes, class_names, ax1, title="Custom CNN")    
plot_heatmap(true_classes, pred_classes, class_names, ax1, title="Transfer Learning (VGG16) No Fine-Tuning")    
# plot_heatmap(true_classes, vgg_pred_classes_ft, class_names, ax3, title="Transfer Learning (VGG16) with Fine-Tuning")    

# fig.suptitle("Confusion Matrix Model Comparison", fontsize=24)
fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()